In [1]:
import numpy as np
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.realestate.com.au/international/us/rent'

browser = webdriver.Chrome()
browser.get(url)
browser.implicitly_wait(20)

In [3]:
houses = pd.DataFrame(columns=[
    'Price', 
    'Rooms',
    'Type',
    'Status',
    'Parking Spaces',
    'Land Size',
    'Building Size',
    'Architecture Style', 
    'County',
    'Provider',
])

In [4]:
while len(houses) < 4000:
    links = browser.find_elements(By.XPATH, '//div[@class="sc-1dun5hk-0 cOiOrj"]/a')
    urls = [link.get_attribute('href') for link in links]
    
    for sub_url in urls:
        browser.execute_script("window.open(arguments[0], '_blank');", sub_url)
        browser.switch_to.window(browser.window_handles[1]) 
        
        try:
            price_tag = browser.find_element(By.XPATH, '//div[@class="sc-10v3xoh-1 cqrlhJ"]')
            price = price_tag.text
        except NoSuchElementException:
            price = None
        
        try:
            info_list = browser.find_elements(By.XPATH, '//div[contains(@class, "sc-12iqlu8-0")]')
            
            basic_info = {}
            for info in info_list:
                key = info.find_element(By.XPATH, './/div[@class="sc-12iqlu8-1 Mguuo basicInfoKey"]').text
                value = info.find_element(By.XPATH, './/div[@class="sc-12iqlu8-2 dPeBQf basicInfoValue"]').text
                basic_info[key] = value
            
            house = pd.Series([
                price,
                basic_info.get('Rooms', None),
                basic_info.get('Property Type', None),
                basic_info.get('Listing Status', None),
                basic_info.get('Parking Spaces', None),
                basic_info.get('Land Size', None),
                basic_info.get('Building Size', None),
                basic_info.get('Architecture Style', None),
                basic_info.get('County', None),
                basic_info.get('Provider', None)
            ], index=houses.columns)
            
            houses.loc[len(houses)] = house
        except Exception as e:
            print(f"Error occurred while scraping {sub_url}: {e}")
        
        browser.close()
        browser.switch_to.window(browser.window_handles[0])

    try:
        next_page = browser.find_element(By.XPATH, '//li[@title="Next Page"]/a')
        next_url = next_page.get_attribute('href')
        browser.get(next_url)
    except NoSuchElementException:
        break

browser.quit()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//li[@title="Next Page"]/a"}
  (Session info: chrome=130.0.6723.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6A2D43AB5+28005]
	(No symbol) [0x00007FF6A2CA83B0]
	(No symbol) [0x00007FF6A2B4580A]
	(No symbol) [0x00007FF6A2B95A3E]
	(No symbol) [0x00007FF6A2B95D2C]
	(No symbol) [0x00007FF6A2BDEA97]
	(No symbol) [0x00007FF6A2BBBA7F]
	(No symbol) [0x00007FF6A2BDB8B3]
	(No symbol) [0x00007FF6A2BBB7E3]
	(No symbol) [0x00007FF6A2B875C8]
	(No symbol) [0x00007FF6A2B88731]
	GetHandleVerifier [0x00007FF6A303643D+3118829]
	GetHandleVerifier [0x00007FF6A3086C90+3448640]
	GetHandleVerifier [0x00007FF6A307CF0D+3408317]
	GetHandleVerifier [0x00007FF6A2E0A40B+841403]
	(No symbol) [0x00007FF6A2CB340F]
	(No symbol) [0x00007FF6A2CAF484]
	(No symbol) [0x00007FF

In [5]:
houses.to_csv('houses.csv', index=False)